# Development of Wastewater Surveillance Data Automation Script 

## 1) Export All data from LIMS DatabasE

In [1]:
#run script that executes export of LIMS data

%run -i "viral_lims_export.py"

In [ ]:
df = export_df_from_LIMS()
df.info()

# Appendix 

## A-I) Explore datatype stored in LIMS database

pyodbc cursor object allows to interact with database parameters. cursos.columns() returns information about every column in the database table.

In [ ]:
cnxn = pyodbc.connect(credentials) # credentials = 'DSN=LIMS_DATA;UID=xxxxxxx;PWD=xxxxxxx'
cursor = cnxn.cursor()

dtype_list = [(i.column_name, i.type_name) for i in cursor.columns(table="vz_Epi_ELS_SARS-CoV-2 ddPCR")]

dtype_list


RESULT: Two columns have datetime type, remaining columns are varchar type
('TestResultDate', 'datetime')
('SampleCollectDate', 'datetime')

## A-II) Explore converting LIMS dataframe to numeric type - may not be necessary.

In [ ]:
potential_numeric = ["NumNoTargetControl", "SARSCoV2AvgConc"]

In [ ]:
df[potential_numeric] = df[potential_numeric].apply(pd.to_numeric, errors = "coerce")

In [ ]:
df.info()

## B-I) REDCap Manual data export

Exploring manual csv data export - column ID's, Datatypes, Exporting Survey ID and Survey Timestamp

**Conclusion**: 2 additional column are present in when manually exporting csv and keeping survey ID and Survey timestamp selected

In [2]:
import pandas as pd

#import data
df_PID177_manual = pd.read_csv("./redcap_manual_export/PID177_ww_labs.csv")
df_PID177_manual_noID_noTimeStamp = pd.read_csv("./redcap_manual_export/PID177_ww_labs_minus_SurTimestamp_SurIdentifier.csv")

#make set of column names
columns_PID177_full = set(df_PID177_manual.columns)
columns_PID177_minimal = set(df_PID177_manual_noID_noTimeStamp.columns)

#compare column sets
print("additional columns present: " + str(columns_PID177_full - columns_PID177_minimal))
#print(labs_set_minimal - labs_set_full) #returns empty set 

print("\n")
print(df_PID177_manual[['redcap_survey_identifier', 'a3_ww_lab_set_up_timestamp']])
print("\n")
print(df_PID177_manual.info())

additional columns present: {'a3_ww_lab_set_up_timestamp', 'redcap_survey_identifier'}


   redcap_survey_identifier a3_ww_lab_set_up_timestamp
0                       NaN                        NaN
1                       NaN              9/1/2021 9:00
2                       NaN            9/17/2021 15:15
3                       NaN                        NaN
4                       NaN                        NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ww_lab_id                   5 non-null      object 
 1   redcap_survey_identifier    0 non-null      float64
 2   a3_ww_lab_set_up_timestamp  2 non-null      object 
 3   lab_name                    5 non-null      object 
 4   ww_lab_setup_date           1 non-null      object 
 5   street_add                  3 non-null      object 
 6   zipcode          

## B-II) REDCap API Data Export

Explore data export via native REDCap API pull

**Conclusion**: API call return data without the additional columns: [redcap_survey_identifier, a3_ww_lab_set_up_timestamp]. These columns can be pulled when exporting data manualy by checking a box.

**Conclusion**: API export columns and manual export columns are identical when survey_identifier and survey_timestamp field remain uncheck during manual export. 

**Conclusion**: During API export, all column fields are objects. Manual export to csv and load to pandas, yields some numeric fields.

**Conclusion**: Datetime format is different between API export, and manual csv export. 

In [43]:
#export PID177 all data via API
df_PID177_API = redcap_API_export(redcap_api_url, redcap_tokens_prod["PID177"])

API_columns_set = set(df_PID177_API.columns)

#comparing columns of csv manual export with identifier and timestamp fields with standrad API export
print("additional columns present: " + str(columns_PID177_full - API_columns_set))
#print(API_columns_set - columns_PID177_full) #empty set
print()
#are all the columns identical? 
print("Are all the columns identical between standard csv export and API export?")
print(all(df_PID177_manual_noID_noTimeStamp.columns == df_PID177_API.columns))

print()
#converting both manually pulled csv and API data to numberic datatypes (if possible)
df_PID177_API = df_PID177_API.apply(pd.to_numeric, errors = "ignore")
df_PID177_manual_noID_noTimeStamp = df_PID177_manual_noID_noTimeStamp.apply(pd.to_numeric, errors = "ignore")

#converting timestamp 
df_PID177_API["ww_lab_setup_date"] = pd.to_datetime(df_PID177_API["ww_lab_setup_date"])
df_PID177_manual_noID_noTimeStamp["ww_lab_setup_date"] = pd.to_datetime(df_PID177_manual_noID_noTimeStamp["ww_lab_setup_date"])
print("after converting all columns to numeric, and 'ww_lab_setup_date' columns to datetime, are the dataframes identical?")

print(df_PID177_manual_noID_noTimeStamp.equals(df_PID177_manual_noID_noTimeStamp))


additional columns present: {'a3_ww_lab_set_up_timestamp', 'redcap_survey_identifier'}

Are all the columns identical between standard csv export and API export?
True

after converting all columns to numeric, and 'ww_lab_setup_date' columns to datetime, are the dataframes identical?
True
